<a href="https://colab.research.google.com/github/tanyagamayunova/au_recovery_efficiency/blob/main/table_concat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Добавление недостающих строк в таблицу
####Задачи:
1. Определить каких файлов не хватает, и дозаполнить таблицу 2 недостающими значениями.
2. Для внесённых значений проставить в колонке checked значение No
3. В колонке priority: если размер файла меньше 10 мб, то проставить low, если размер файла 10-15 мб - medium, если больше 15 мб,
 значение high

####Подготовка данных

In [34]:
from google.colab import drive
import pandas as pd
import numpy as np

In [35]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
table_1 = pd.read_csv('https://docs.google.com/spreadsheets/d/11jmYU1ogYvRvJojZcLRbDLOqpQ21bubThS2dFn57P24/export?format=csv')
table_2 = pd.read_csv('https://docs.google.com/spreadsheets/d/1BLqBgeD9l0LNC_im4xKTN_XQm9L7CufAqD3jy0il5Uo/export?format=csv')

In [59]:
table_1.info()
display(table_1.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   file                    70 non-null     object 
 1   type                    70 non-null     object 
 2   size                    70 non-null     float64
 3   last modification date  70 non-null     object 
dtypes: float64(1), object(3)
memory usage: 2.3+ KB


,file,type,size,last modification date
0,Storyboard,sound,1.9,7/4/2018
1,Eclipse,picture,3.1,6/23/2018
2,Maelstrom,picture,4.4,7/12/2018
3,Fact,text,4.5,6/21/2018
4,Geese,picture,4.8,5/31/2018
5,Premeditation,sound,5.6,6/12/2018
6,View,text,6.4,7/10/2018
7,forest,picture,6.6,8/15/2018
8,Fact,sound,6.9,7/3/2018
9,Councilperson,text,6.9,8/1/2018


In [60]:
table_2.info()
display(table_2.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   file      65 non-null     object
 1   type      65 non-null     object
 2   priority  65 non-null     object
 3   checked   65 non-null     object
dtypes: object(4)
memory usage: 2.2+ KB


,file,type,priority,checked
0,Aardvark,text,high,No
1,Analysis,sound,medium,No
2,anthem,text,high,Yes
3,anthem,sound,medium,No
4,biology,text,low,No
5,sea,sound,low,Yes
6,Bend,sound,medium,Yes
7,Strain,text,low,No
8,Fertilizer,sound,low,Yes
9,Blossom,sound,medium,No


####Задача 1. Соединение таблиц

Можно объединить таблицы и избавиться от дубликатов по двум колонкам выражением в одну строчку.

In [48]:
table_new = pd.concat((table_2, table_1)).drop_duplicates(subset=['file', 'type'])

In [49]:
table_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70 entries, 0 to 52
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   file                    70 non-null     object 
 1   type                    70 non-null     object 
 2   priority                65 non-null     object 
 3   checked                 65 non-null     object 
 4   size                    5 non-null      float64
 5   last modification date  5 non-null      object 
dtypes: float64(1), object(5)
memory usage: 3.8+ KB


In [52]:
table_new.head(len(table_new.index))

,file,type,priority,checked,size,last modification date
0,Aardvark,text,high,No,NaN,NaN
1,Analysis,sound,medium,No,NaN,NaN
2,anthem,text,high,Yes,NaN,NaN
3,anthem,sound,medium,No,NaN,NaN
4,biology,text,low,No,NaN,NaN
...,...,...,...,...,...,...
15,Protection,picture,NaN,NaN,10.0,6/22/2018
20,Racing,sound,NaN,NaN,14.0,7/18/2018
28,Playground,picture,NaN,NaN,19.8,6/8/2018
46,birds,sound,NaN,NaN,31.1,5/29/2018


####Задача 2. Заполнение колонки checked.

In [53]:
table_new.loc[table_new['checked'].isna(), 'checked'] = 'No'

In [54]:
table_new.tail(5)

,file,type,priority,checked,size,last modification date
15,Protection,picture,NaN,No,10.0,6/22/2018
20,Racing,sound,NaN,No,14.0,7/18/2018
28,Playground,picture,NaN,No,19.8,6/8/2018
46,birds,sound,NaN,No,31.1,5/29/2018
52,Mint,text,NaN,No,34.2,6/18/2018


####Задача 3. Выставление приоритета на основе размера файла.

In [55]:
table_new.loc[table_new['priority'].isna(), 'priority'] = np.where(
    table_new.loc[table_new['priority'].isna(), 'size'] < 10,
    'low',
    np.where(
        table_new.loc[table_new['priority'].isna(), 'size'] > 15,
        'high',
        'medium'
    )
)

In [56]:
table_new.tail(5)

,file,type,priority,checked,size,last modification date
15,Protection,picture,medium,No,10.0,6/22/2018
20,Racing,sound,medium,No,14.0,7/18/2018
28,Playground,picture,high,No,19.8,6/8/2018
46,birds,sound,high,No,31.1,5/29/2018
52,Mint,text,high,No,34.2,6/18/2018


#### Удаление ненужных столбцов и просмотр таблицы.


In [57]:
table_new = table_new.drop(['size', 'last modification date'], axis=1).reset_index(drop=True)

In [61]:
display(table_new.head(len(table_new.index)))

,file,type,priority,checked
0,Aardvark,text,high,No
1,Analysis,sound,medium,No
2,anthem,text,high,Yes
3,anthem,sound,medium,No
4,biology,text,low,No
...,...,...,...,...
65,Protection,picture,medium,No
66,Racing,sound,medium,No
67,Playground,picture,high,No
68,birds,sound,high,No
